In [1]:

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

#from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
#from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#from imblearn.under_sampling import TomekLinks
#from imblearn.under_sampling import ClusterCentroids
#from imblearn.over_sampling import SMOTE
#from imblearn.combine import SMOTETomek

#from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier



In [2]:
#tf = pd.read_csv('featuresMagui.csv')
tf_seba = pd.read_csv('features_seba.csv')
tf_santi = pd.read_csv('santi_timefeatures.csv')
tf_santi_2 = pd.read_csv('Santi_FeaturesConEventos.csv')
labels = pd.read_csv('../../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [3]:
#Datos a entregar
santi_seba = pd.merge(tf_santi_2,tf_seba, on = 'person',how = 'inner')
datos = pd.merge(personas, santi_seba, on = 'person', how = 'inner')

In [4]:
#Datos para entrenar
#labels_f = pd.merge(tf_santi_2,tf_seba, on = 'person',how = 'inner')
labels_f = pd.merge(labels, santi_seba, on = 'person', how = 'inner')

In [5]:
y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 406]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 123)

# Seleccion de features con RFE

### Usando todos los features

In [7]:
from sklearn.feature_selection import RFE
#Instanciamos el regresor de XGBoost

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)


In [8]:
n = 43
rfe = RFE(xg_reg, n)
rfe = rfe.fit(X, y['label'].ravel())
   
print('n = ', n)
   
# Selecciono los n features para entrenar
mask = rfe.get_support()
features_X = X.columns[mask]
new_x = X.filter(items=features_X)

n =  43


In [9]:
# Entrenamos y predecimos
X_train, X_test, y_train, y_test = train_test_split(new_x, y['label'], test_size=0.2, random_state=123)
    
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(X_train,y_train)
proba = xg_reg.predict_proba(X_test)[:,1]
print("ROC auc score: ", roc_auc_score(y_test, proba))
# con tf_santi_2,tf_seba, y n=43 me da 0.8626966186972823 (Filtrando X) ESTE SUBI A KAGGLE Y DIO 0.86148

ROC auc score:  0.8626966186972823


#### Para subir a Kaggle

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(new_x,y['label'])


In [ ]:
df_final = datos.filter(items= ['person'] + list(features_X))
df_final_sin_persons= df_final.drop(columns='person')
df_final.head(10)

In [ ]:
proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

predicciones = df_final[['person']]

predicciones['label'] = proba

In [ ]:
predicciones.to_csv('predicciones_kaggle.csv', encoding='utf-8', index=False)

In [ ]:
predicciones.shape

In [ ]:
predicciones.head()

# Seleccion de features con random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)
preds_prob = clf.predict_proba(X_test)[:,1]
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
matriz_de_confusion = confusion_matrix(y_test, preds)
print('train acurracy: ')
print(train_accuracy)
print('test acurracy: ')
print(test_accuracy)
print('Matriz de confusión: ')
print(matriz_de_confusion)
print('Área bajo la curva: ')
print(area_debajo_de_curva)

In [ ]:
importancia = clf.feature_importances_
len(importancia)

In [ ]:
importancia_con_nombre = {}
features = X.columns
for i in range(193):
    importancia_con_nombre[features[i]] = importancia[i]
importancia_con_nombre

In [ ]:
atributos_importantes = []
for clave in importancia_con_nombre.keys():
    if importancia_con_nombre[clave] > 0:
        atributos_importantes.append(clave)

In [ ]:
len(atributos_importantes)

In [ ]:
atributos_importantes

In [ ]:
X_imp = X.filter(items= atributos_importantes)
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size= 0.2, random_state= 123)

### Pruebo con XGBoost

In [ ]:
def xgboost(X_p, y_p, X_train_p, X_test_p, y_train_p, y_test_p):
    xg_reg = xgb.XGBClassifier(learning_rate =0.1, n_estimators=6, max_depth=5, min_child_weight=12,
                           colsample_bytree=0.8, objective= 'binary:logistic', scale_pos_weight = 5)
    
    xg_reg.fit(X_train_p,y_train_p, early_stopping_rounds = 5, eval_set=[(X_test, y_test)])
    preds = xg_reg.predict(X_test_p)
    preds_prob = xg_reg.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, xg_reg.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    return(xg_reg)

In [ ]:
xg_reg =xgboost(X_imp, y, X_train, X_test, y_train, y_test)

# Seleccion de features con RFECV

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=xg_reg, step=1, cv=StratifiedKFold(10),
              scoring='roc_auc')
rfecv.fit(X, y['label'].ravel())

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
# Selecciono los n features para entrenar
mask = rfecv.get_support()
features_X = X.columns[mask]
new_x = X.filter(items=features_X)

In [ ]:
# Entrenamos y predecimos
X_train, X_test, y_train, y_test = train_test_split(new_x, y['label'], test_size=0.2, random_state=123)
    
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(X_train,y_train)
proba = xg_reg.predict_proba(X_test)[:,1]
print("ROC auc score: ", roc_auc_score(y_test, proba))

### Con sampleo

In [ ]:
def kaggle_oversampling(df, cant_unos):
   
    df_train = pd.merge(df, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
    # Separamos la variable a predecir

    y = labels_f.iloc[:,1:2]
    X = labels_f.iloc[:,3: 406]
    
    datos = pd.merge(df, personas, on='person', how='inner').filter(items= list(df_train.columns) + ['person'])
    df_final_sin_persons = datos.drop(columns='person')
     
    cant_ceros = np.sum(y == 0)
    
    ros = RandomOverSampler(random_state=2, sampling_strategy = {0: cant_ceros, 1: cant_unos})
    X_res, y_res = ros.fit_sample(X, y)
    X_res_df = pd.DataFrame(X_res)
    X_res_df.columns = X_test.columns

    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    #Entrenamos

    xg_reg.fit(X_res_df, y_res)
    
    proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

    predicciones = datos[['person']]
    predicciones['label'] = proba
    predicciones.to_csv('predicciones.csv', encoding='utf-8', index=False)

    print(predicciones.shape[0] == 19415)

    train_accuracy = accuracy_score(y_res, xg_reg.predict(X_res_df))

    print("Train accuracy: ", train_accuracy)


#### Para subir a Kaggle

In [ ]:
cant_ceros = np.sum(y == 0)

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1191})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train.ravel())



In [ ]:

xg_reg.fit(Xr_df, yr['label'])


In [ ]:
df_final = datos.filter(items= ['person'] + list(features_X))
df_final_sin_persons= df_final.drop(columns='person')
df_final.head(10)

In [ ]:
#preds = xg_reg.predict(X_test)
proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

predicciones = df_final[['person']]

predicciones['label'] = proba

In [ ]:
predicciones.to_csv('predicciones_oversamp.csv', encoding='utf-8', index=False)

In [ ]:
predicciones.shape

In [ ]:
predicciones.head()

In [ ]:
predicciones.head()

In [ ]:
def probar_oversampling(df, cant_unos):
    
    labels_f = pd.merge(labels, df, on = 'person', how = 'inner')
    y = labels_f.iloc[:,1:2]
    X = labels_f.iloc[:,3: 406]
    
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)
    n = 22
    rfe = RFE(xg_reg, n)
    rfe = rfe.fit(X, y['label'].ravel())
   
    print('n = ', n)
   
    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    new_x = X.filter(items=features_X)
    

    X_train, X_test, y_train, y_test = train_test_split(new_x, y, test_size=0.2, random_state=123)
    
    cant_ceros = np.sum(y_train == 0)
    
    ros = RandomOverSampler(random_state=2, sampling_strategy = {0: cant_ceros, 1: cant_unos})
    X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
    X_train_res_df = pd.DataFrame(X_train_res)
    X_train_res_df.columns = X_test.columns


    xg_reg.fit(X_train_res_df, y_train_res)

    preds = xg_reg.predict(X_test)
    proba = xg_reg.predict_proba(X_test)[:,1]
    
    train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
    test_accuracy = accuracy_score(y_test, preds)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, proba)

    print("Train accuracy: ", train_accuracy)
    print("Test acuracy: ", test_accuracy)
    print("ROC auc score: ", area_debajo_de_curva)
    print("Confusion matrix: ")
    print(matriz_de_confusion)

#### iterando

In [ ]:
# Con el seba_features.csv
for n in range(40,100,5):
    # create the RFE model and select n attributes
    rfe = RFE(xg_reg, n)
    rfe = rfe.fit(X, y['label'].ravel())
    
    print('n = ', n)
    
    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    new_x = X.filter(items=features_X)
    
    # Entrenamos y predecimos
    X_train, X_test, y_train, y_test = train_test_split(new_x, y['label'], test_size=0.2, random_state=123)
    
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    xg_reg.fit(X_train,y_train)
    proba = xg_reg.predict_proba(X_test)[:,1]
    print("ROC auc score: ", roc_auc_score(y_test, proba))


In [ ]:
# Con el seba_features.csv
for n in range(85,90,1):
    # create the RFE model and select n attributes
    rfe = RFE(xg_reg, n)
    rfe = rfe.fit(X, y['label'].ravel())
    
    print('n = ', n)
    
    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    new_x = X.filter(items=features_X)
    
    # Entrenamos y predecimos
    X_train, X_test, y_train, y_test = train_test_split(new_x, y['label'], test_size=0.2, random_state=123)
    
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    xg_reg.fit(X_train,y_train)
    proba = xg_reg.predict_proba(X_test)[:,1]
    print("ROC auc score: ", roc_auc_score(y_test, proba))


Funciones CV

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict

In [ ]:
def stratifiedCVConRFE(data, n):
    
    df_train = pd.merge(data, labels,how = 'inner', on ='person')
    df_train.drop(columns = ['person'],inplace = True)
    
    X = df_train.drop(columns = ['label'])
    y = df_train['label']
    
    model = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)
    
    stratifiedKFold = StratifiedKFold(n_splits = 10, random_state = 7)
    
    #n = 22
    rfe = RFE(model, n)
    rfe = rfe.fit(X, y.ravel())

    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    X = X.filter(items = features_X)
    
    results = cross_val_score(model, X, y, cv = stratifiedKFold, scoring = 'roc_auc')
        
    print("ROC AUC: ",results.mean())
    return results.mean()

In [ ]:
def stratifiedCV(data):
    
    df_train = pd.merge(data,df_labels,how = 'inner', on ='person')
    df_train.drop(columns = ['person'],inplace = True)
    
    X = df_train.drop(columns = ['label'])
    y = df_train['label']
    
    model = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)
    
    stratifiedKFold = StratifiedKFold(n_splits = 10, random_state = 7)
    
    results = cross_val_score(model, X, y, cv = stratifiedKFold, scoring = 'roc_auc')
    #results2 = cross_val_predict(model, X, y, cv = stratifiedKFold, method='predict_proba')
    
    print("ROC AUC: ",results.mean())

In [ ]:
stratifiedCVConRFE(santi_seba)

In [ ]:
#n= 43 da 0.8557534566892058
stratifiedCVConRFE(santi_seba)

#### Iterando con CV y RFE para obtener el mas optimo

In [ ]:
bestK = 0
score = 0
for i in range(17,193):
    score_act = stratifiedCVConRFE(santi_seba, i)
    if(score_act > score):
        score = score_act
        bestK = i
        print(bestK)

### Usando los features ''importantes'' del random forest

In [ ]:
n = 43
rfe = RFE(xg_reg, n)
rfe = rfe.fit(X_imp, y['label'].ravel())
   
print('n = ', n)
   
# Selecciono los n features para entrenar
mask = rfe.get_support()
features_X = X_imp.columns[mask]
new_x = X_imp.filter(items=features_X)

In [ ]:
# Entrenamos y predecimos
X_train, X_test, y_train, y_test = train_test_split(new_x, y['label'], test_size=0.2, random_state=123)
    
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(X_train,y_train)
proba = xg_reg.predict_proba(X_test)[:,1]
print("ROC auc score: ", roc_auc_score(y_test, proba))
# con tf_santi_2,tf_seba, y n=43 me da 0.8670756921715554 (con X_imp)

#### Para subir a Kaggle

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(new_x,y['label'])


In [ ]:
df_final = datos.filter(items= ['person'] + list(features_X))
df_final_sin_persons= df_final.drop(columns='person')
df_final.head(10)

In [ ]:
#preds = xg_reg.predict(X_test)
proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

predicciones = df_final[['person']]

predicciones['label'] = proba

In [ ]:
predicciones.to_csv('predicciones_p5.csv', encoding='utf-8', index=False)

In [ ]:
predicciones.shape

In [ ]:
predicciones.head()

### Para subir a kaggle con los features 'importantes' de rf

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

xg_reg.fit(new_x,y['label'])


In [ ]:
df_final = datos.filter(items= ['person'] + list(features_X))
df_final_sin_persons= df_final.drop(columns='person')
df_final.head(10)

In [ ]:
#preds = xg_reg.predict(X_test)
proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

predicciones = df_final[['person']]

predicciones['label'] = proba

In [ ]:
predicciones.to_csv('predicciones_p3.csv', encoding='utf-8', index=False)

In [ ]:
predicciones.head()